In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.co

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-20 19:38:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.24MB/s    in 0.2s    

2022-10-20 19:38:13 (5.24 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://<MYBUCKET>.s3.amazonaws.com/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [7]:
# Load in a function to use columns
from pyspark.sql.functions import col

# Filter for only columns with total_votes equal or greater than 20
helpful_votes_df = df.filter(col("total_votes") >= 20)
helpful_votes_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43836009|R1CY3APKBOVUYU|B00VO8D13K|      82685655|         Citizenfour|Digital_Video_Dow...|          1|            9|         33|   N|                Y|Unnecessarily Slo...|This is an unnece...| 2015-08-31|
|         US|   16033254|R2LTI28AN2281A|B00YOSAJBO|     463297585|  Merchants Of Doubt|Digital_Video_Dow

In [8]:
# Retrieve rows where helpful_votes/total_votes is >= 50%
very_helpful_votes_df = helpful_votes_df.filter(col("helpful_votes")/col("total_votes") >= .50)
very_helpful_votes_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    6822472| RZM83AGJAUZMA|B0142KIQYA|     273405592|I'll See You in M...|Digital_Video_Dow...|          1|           32|         41|   N|                Y|A Conscious Uncou...|if anyone finds t...| 2015-08-31|
|         US|   53001773|R12MYB9QM2YN8B|B00OLI95R6|     955620049|                Fury|Digital_Video_Dow

In [9]:
# Filter only reviews written as part of the Vine program
vine_reviews_df = very_helpful_votes_df.filter(col("vine") == 'Y')
vine_reviews_df.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [10]:
# Filter only reviews not written as part of the Vine program
non_vine_reviews_df = very_helpful_votes_df.filter(col("vine") == 'N')
non_vine_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    6822472| RZM83AGJAUZMA|B0142KIQYA|     273405592|I'll See You in M...|Digital_Video_Dow...|          1|           32|         41|   N|                Y|A Conscious Uncou...|if anyone finds t...| 2015-08-31|
|         US|   53001773|R12MYB9QM2YN8B|B00OLI95R6|     955620049|                Fury|Digital_Video_Dow

In [17]:
# Determine the total number of reviews
vine_reviews_df.count()
print("Total number of reviews for the paid Vine program was: %f" % vine_reviews_df.count())
non_vine_reviews_df.count()
print("Total number of reviews for the unpaid Vine program was: %f" % non_vine_reviews_df.count())
total_vine_reviews = vine_reviews_df.count() + non_vine_reviews_df.count()
print("Total number of Vine reviews was: %f" % total_vine_reviews)

# Determine the number of paid 5-star reviews
paid_5stars_df = vine_reviews_df.filter(col("star_rating") == 5)
print("Number of paid 5-star reviews was: %f" % paid_5stars_df.count())

# Determine thenumber of unpaid 5-star reviews
unpaid_5stars_df = non_vine_reviews_df.filter(col("star_rating") == 5)
print("Number of unpaid 5-star reviews was: %f" % unpaid_5stars_df.count())

# Determine the total number of 5-star Vine reviews
fivestars = paid_5stars_df.count() + unpaid_5stars_df.count()
print("Total number of 5-star Vine reviews was: %f" % fivestars)

# Determine the % of paid 5-star reviews vs. total reviews
paid_5stars_share = paid_5stars_df.count() * 100 / total_vine_reviews
print("Share of paid 5-star reviews was: %f" % paid_5stars_share + "% of total reviews")

# Determine the % of unpaid 5-star reviews vs. total reviews
unpaid_5stars_share = unpaid_5stars_df.count() * 100 / total_vine_reviews
print("Share of unpaid 5-star reviews was: %f" % unpaid_5stars_share + "% of total reviews")

# Determine the % of paid 5-star reviews vs. total 5-star reviews
paid_fivestars_share = paid_5stars_df.count() * 100 / fivestars
print("Share of paid 5-star reviews was: %f" % paid_fivestars_share + "% of 5-star reviews")

# Determine the % of unpaid 5-star reviews vs. total 5-star reviews
unpaid_fivestars_share = unpaid_5stars_df.count() * 100 / fivestars
print("Share of unpaid 5-star reviews was: %f" % unpaid_fivestars_share + "% of 5-star reviews")

Total number of reviews for the paid Vine program was: 0.000000
Total number of reviews for the unpaid Vine program was: 11371.000000
Total number of Vine reviews was: 11371.000000
Number of paid 5-star reviews was: 0.000000
Number of unpaid 5-star reviews was: 4675.000000
Total number of 5-star Vine reviews was: 4675.000000
Share of paid 5-star reviews was: 0.000000% of total reviews
Share of unpaid 5-star reviews was: 41.113359% of total reviews
Share of paid 5-star reviews was: 0.000000% of 5-star reviews
Share of unpaid 5-star reviews was: 100.000000% of 5-star reviews
